#### imports

In [1]:
import pandas as pd
import numpy as np
import polars as pl

### strcuture of different datasets

#### df_channels_en.tsv

CSV file, having the following structure:
| **category_cc** | **join_date** | **channel**              | **name_cc**             | **subscribers_cc** | **videos_cc** | **subsriber_rank_sb** | **weights** |
|-----------------|---------------|--------------------------|-------------------------|--------------------|---------------|-----------------------|-------------|
| News & Politics | 2013-03-11    | UCcRgZlgsk5m-aDQa_d6BTkQ | NorthWestLibertyNews... | 16700              | 845           | 639043.0              | 10.0035     |
| Gaming          |    2012-01-15 | UCnnXR0VIJVpeL1wEr-bBaRw | Felix Guaman            | 112000             | 703           | 137318.0              | 5.4915      |

With:
- `category_cc`: most frequent category of the channel. One of: ['Gaming', 'Education', 'Entertainment', 'Howto & Style', 'Sports', 'Music', 'Film and Animation', 'Comedy', 'Nonprofits & Activism', 'People & Blogs', 'News & Politics', 'Science & Technology', 'Pets & Animals', 'Autos & Vehicles', 'Travel & Events', nan]
- `join_date`: join date of the channel.
- `channel`: unique channel id.
- `name_cc`: name of the channel.
- `subscribers_cc`: number of subscribers.
- `videos_cc`: number of videos.
- `subscriber_rank_sb`: rank in terms of number of subscribers.
- `weights`: weights cal



#### yt_metadata_helper.feather   (yt_metadata_helper.feather.csv, filtered_yt_metadata_helper.feather.csv)
<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th>categories</th>
      <th>channel_id</th>
      <th>dislike_count</th>
      <th>display_id</th>
      <th>duration</th>
      <th>like_count</th>
      <th>upload_date</th>
      <th>view_count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Film &amp; Animation</td>
      <td>UCy6sWF4taso5GtrfDGhwpBA</td>
      <td>0.0</td>
      <td>EXOviJ_EJDo</td>
      <td>68</td>
      <td>0.0</td>
      <td>2011-12-07</td>
      <td>76.0</td>
    </tr>
    <tr>
      <td>Gaming</td>
      <td>UCEPYwwuGhgA9wfO2It11OXw</td>
      <td>0.0</td>
      <td>xSKA6VX7Tdo</td>
      <td>125</td>
      <td>6.0</td>
      <td>2016-10-04</td>
      <td>198.0</td>
    </tr>
    <tr>
      <td>News &amp; Politics</td>
      <td>UCojNA7ZvnmGuIvYnm44wl3Q</td>
      <td>NaN</td>
      <td>FsucWMijKA4</td>
      <td>130</td>
      <td>NaN</td>
      <td>2010-11-18</td>
      <td>106.0</td>
    </tr>
  </tbody>
</table>
</div>

With (values were crawled from YouTube between 2019-10-29 and 2019-11-23):
- `categories`: category (self-defined when they upload a video to YouTube)
- `channel_id`: unique channel id
- `dislike_count`: dislikes of the video
- `display_id`: unique video id
- `duration`: duration of the video
- `like_count`:likes of the video.
- `upload_date`: upload date
- `view_count`: views of the video.

#### df_timeseries_en.tsv

| **channel**              | **category**       | **datetime** | **views**   | **delta_views** | **subs** | **delta_subs** | **videos** | **delta_videos** | **activity** |
|--------------------------|--------------------|--------------|-------------|-----------------|----------|----------------|------------|------------------|--------------|
| UCBJuEqXfXTdcPSbGO9qqn1g | Film and Animation | 2017-07-03 | 202495  |           0 |  650 |   0        |      5 |            0 |        3 |
| UCBJuEqXfXTdcPSbGO9qqn1g | Film and Animation | 2017-07-10 | 394086  |      191591 | 1046 | 396        |      6 |            1 |        1 |

With:
- `channel`: channel id.
- `category`: category of the channel as assigned by `socialblade.com` according to the last 10 videos at time of crawl.
- `datetime`: Week related to the data point.
- `views`: Total number of views the channel had this week.
- `delta_views`: Delta views obtained this week.
- `subs`: Total number of subscribers the channel had this week.
- `delta_subs`: Delta subscribers obtained this week.
- `videos`: Total number of videos the channel had this week.
- `delta_videos`: Delta videos obtained this week.
- `activity`: Number of videos published in the last 15 days.


#### youtube_comments.tsv

| **author** | **video_id**      |  **likes** |  **replies** |
|------|--------------|-------|---------|
| 1      | Gkb1QMHrGvA   |  2     |  0       |
| 1      | CNtp0xqoods   |  0     |  0       |
| 1      | 249EEzQmVmQ   |  1     |  0       |

With (data obtained at crawl time between 2019-09-12 and 2019-09-17):
- `author`: anonymized author id (unique)
- `video_id`: unique video id of the video the comment was written
- `likes`: likes for the comment
- `replies`: replies for the comment

In [2]:
df_ch_f = pd.read_csv("../Youniverse/df_channels_en.tsv.gz", compression="infer", sep="\t")

pl_df_f = pl.read_csv("../Youniverse/df_channels_en.tsv", separator="\t")

filtered_df_ch = pl_df_f.filter(pl.col("category_cc") == "News & Politics")
pl_df_f.sample(5)

category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
str,str,str,str,i64,i64,f64,f64
"""People & Blogs""","""2014-04-27""","""UCygbRxfq8g4GCmHZAN5Aolw""","""BeastWithAesthetics""",23526,447,511586.0,8.9545
"""Comedy""","""2006-07-06""","""UCguU4Tey59bTXw98WmeVICA""","""Willis Clinton Harri...""",14100,92,494028.0,7.6945
"""People & Blogs""","""2013-08-13""","""UCcbBHQXuSkw8Me1vnY-u4bw""","""Prophet Shepherd Bus...""",353000,1777,45863.0,3.285
"""Entertainment""","""2014-10-08""","""UC2SyTi1cPbBJWR-StNbI01Q""","""ZiGGy Toys""",104000,60,162805.0,5.461
"""Film and Animation""","""2006-05-09""","""UCgY-h0zGG5EOa3f_k22wMJg""","""Sainsbury's""",109000,407,145967.0,4.1


In [ ]:
# df_vd_f = pd.read_feather("../Youniverse/yt_metadata_helper.feather")

# df_vd_f.sample(5)

,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count
16583269,Film & Animation,UCy6sWF4taso5GtrfDGhwpBA,0.0,EXOviJ_EJDo,68,0.0,2011-12-07,76.0
51429663,Gaming,UCEPYwwuGhgA9wfO2It11OXw,0.0,xSKA6VX7Tdo,125,6.0,2016-10-04,198.0
4222786,News & Politics,UCojNA7ZvnmGuIvYnm44wl3Q,NaN,FsucWMijKA4,130,NaN,2010-11-18,106.0
61632041,Music,UC6SemiBqCDvv2XsWVqD8mgg,0.0,3PGYujlHr38,290,0.0,2018-06-09,0.0
31333325,Sports,UCTtkVP_v56wU8zXpVBpdaZQ,0.0,BKvxa7AbuMI,263,2.0,2016-08-26,1201.0


In [ ]:
# save to csv
# df_vd_f.to_csv("../Youniverse/yt_metadata_helper.feather.csv", sep="\t")

In [5]:
filtered_df_ch.filter(pl.col("channel").is_in(filtered_df_ch["channel"]))

category_cc,join_date,channel,name_cc,subscribers_cc,videos_cc,subscriber_rank_sb,weights
str,str,str,str,i64,i64,f64,f64
"""News & Politics""","""2006-08-26""","""UCttspZesZIDEwwpVIgoZtWQ""","""IndiaTV""",15177282,139814,199.0,2.087
"""News & Politics""","""2012-06-01""","""UCRWFSbif-RFENbBrSiez1DA""","""ABP NEWS""",16274836,129027,207.0,2.087
"""News & Politics""","""2017-03-03""","""UCmphdqZNmqL72WJ2uyiNw5w""","""ABP NEWS HINDI""",10800000,51298,340.0,2.087
"""News & Politics""","""2015-03-23""","""UCx8Z14PpntdaxCt2hakbQLQ""","""The Lallantop""",9120000,9423,438.0,2.087
"""News & Politics""","""2007-06-19""","""UCIvaYmXn910QMdemBG3v1pQ""","""Zee News""",9280000,102648,549.0,2.087
…,…,…,…,…,…,…,…
"""News & Politics""","""2010-08-07""","""UC5rxiCGcNunIi5zI1hMYLMg""","""Salman Akhtar""",10400,40,962468.0,53.1435
"""News & Politics""","""2013-02-01""","""UCLSEJQ8TWtlEkaytaa4Y7lw""","""WingsOfChrist""",10420,61,962547.0,53.1435
"""News & Politics""","""2012-10-19""","""UCnkG_c5cyemVVsgCDoHiXew""","""The American Mirror""",10500,329,963417.0,53.1435


In [ ]:
# load yt metadata in chunks and filter for videos contained in filtered_df_ch
# df.filter(pl.col("categories") == "News & Politics") to filter for categories of videos instead
reader = pl.read_csv_batched(
    "../Youniverse/yt_metadata_helper.feather.csv",
    separator="\t",
    batch_size=5000
)


batches = reader.next_batches(5)  
i = 0
while batches:
    for df in batches:
        if i == 0:
            df.filter(pl.col("channel_id").is_in(filtered_df_ch["channel"])).write_csv("../Youniverse/filtered_yt_metadata_helper.feather.csv", include_header=True)
        else:
            with open("../Youniverse/filtered_yt_metadata_helper.feather.csv", "a") as fh:
                fh.write(df.filter(pl.col("channel_id").is_in(filtered_df_ch["channel"])).write_csv(file=None, include_header=False))
        i = i+1
        print(f"batch {i}\r", end='')
    batches = reader.next_batches(5)

In [4]:
filtered_df_metadata_feather = pl.read_csv("../Youniverse/filtered_yt_metadata_helper.feather.csv")
filtered_df_metadata_feather.sample(5)

,categories,channel_id,dislike_count,display_id,duration,like_count,upload_date,view_count
i64,str,str,f64,str,i64,f64,str,f64
25924058,"""News & Politics""","""UCYC4ijpFZY_CtdElWFyy-Gg""",626.0,"""u-1i6jC5SpI""",379,1185.0,"""2017-12-20""",68429.0
10858010,"""News & Politics""","""UCj7wKsOBhRD9Jy4yahkMRMw""",3.0,"""fXN0ABkfZ7M""",158,19.0,"""2019-09-05""",622.0
60789420,"""Entertainment""","""UC6ejCxeEjpMIjHRPD8o5prg""",1.0,"""bNn-MU6l5zo""",98,17.0,"""2017-05-25""",987.0
67253323,"""Comedy""","""UC1vOCYEu3DfoI4fYcfWKmWA""",null,"""rGpvBxryv1I""",272,null,"""2019-04-21""",4269.0
52486580,"""News & Politics""","""UCDS9hpqUEXsXUIcf0qDcBIA""",1.0,"""c1x-wWsXfCU""",95,4.0,"""2018-06-27""",172.0


In [18]:
len(filtered_df_metadata_feather)

9503610

In [5]:
list(filtered_df_metadata_feather["categories"].unique())

['Nonprofits & Activism',
 'People & Blogs',
 'Pets & Animals',
 'Howto & Style',
 'Education',
 None,
 'Comedy',
 'Entertainment',
 'Gaming',
 'Autos & Vehicles',
 'Travel & Events',
 'Film & Animation',
 'Sports',
 'News & Politics',
 'Science & Technology',
 'Music']

In [10]:
filtered_df_metadata_feather["categories"].value_counts().sort(by="count", descending=True)

categories,count
str,u32
"""News & Politics""",8261387
"""Entertainment""",398379
"""People & Blogs""",389905
"""Sports""",116509
"""Education""",72252
…,…
"""Comedy""",21957
"""Pets & Animals""",12286
"""Autos & Vehicles""",10680


In [7]:
chunks = pd.read_json("../Youniverse/yt_metadata_en.jsonl", lines=True, chunksize = 500)
for i, c in enumerate(chunks):
    c = c[c["channel_id"].isin(filtered_df_ch["channel"])]
    if i == 0:
        c.to_csv("../Youniverse/filtered_yt_metadata.csv", header=True)
    else: 
        with open("../Youniverse/filtered_yt_metadata.csv", "a") as fh:
            fh.write(c.to_csv(path_or_buf=None, header=False))
    print(f"batch {i} / 145390 \r", end="")

In [9]:
# load yt metadata in chunks and filter for videos contained in filtered_df_ch
# df.filter(pl.col("categories") == "News & Politics") to filter for categories of videos instead
batch_size = 10000
reader = pl.read_csv_batched(
    "../Youniverse/youtube_comments.tsv",
    separator="\t",
    batch_size= batch_size
)

total_batches = 8600000000/batch_size

batches = reader.next_batches(5)  
i = 0
while batches:
    for df in batches:
        if i == 0:
            # df.filter(pl.col("video_id").is_in(filtered_df_metadata_feather["display_id"])).write_csv("../Youniverse/filtered_youtube_comments.tsv", include_header=True)
            df.filter(pl.col("video_id") == "fXN0ABkfZ7M").write_csv("../Youniverse/filtered_youtube_comments.tsv", include_header=True)
        else:
            with open("../Youniverse/filtered_youtube_comments.tsv", "a") as fh:
                fh.write(df.filter(pl.col("video_id") == "fXN0ABkfZ7M").write_csv(file=None, include_header=False))
        i = i+1
        print(f"batch {i} / {total_batches} \r", end='')
    batches = reader.next_batches(5)

KeyboardInterrupt: 

In [ ]:
# load yt metadata in chunks and filter for videos contained in filtered_df_ch
reader = pl.read_csv_batched(
    "../Youniverse/num_comments_authors.tsv",
    separator="\t",
    batch_size=5000
)  
batches = reader.next_batches(5)  
for df in batches:  
    print(df)

shape: (4_600, 4)
┌────────┬─────────────┬───────┬─────────┐
│ author ┆ video_id    ┆ likes ┆ replies │
│ ---    ┆ ---         ┆ ---   ┆ ---     │
│ i64    ┆ str         ┆ i64   ┆ i64     │
╞════════╪═════════════╪═══════╪═════════╡
│ 1      ┆ Gkb1QMHrGvA ┆ 2     ┆ 0       │
│ 1      ┆ CNtp0xqoods ┆ 0     ┆ 0       │
│ 1      ┆ 249EEzQmVmQ ┆ 1     ┆ 0       │
│ 1      ┆ _U443T2K_Bs ┆ 0     ┆ 0       │
│ 1      ┆ rJbjhm0weYc ┆ 0     ┆ 0       │
│ …      ┆ …           ┆ …     ┆ …       │
│ 314    ┆ cvrCYT7Oyt4 ┆ 0     ┆ 0       │
│ 314    ┆ B90EzQj2wOQ ┆ 0     ┆ 0       │
│ 314    ┆ BkHuP2ZEtAY ┆ 0     ┆ 0       │
│ 314    ┆ 9ndXWa3j91I ┆ 0     ┆ 0       │
│ 314    ┆ 9qfNbzqZlBk ┆ 0     ┆ 0       │
└────────┴─────────────┴───────┴─────────┘
shape: (4_490, 4)
┌────────┬─────────────┬───────┬─────────┐
│ author ┆ video_id    ┆ likes ┆ replies │
│ ---    ┆ ---         ┆ ---   ┆ ---     │
│ i64    ┆ str         ┆ i64   ┆ i64     │
╞════════╪═════════════╪═══════╪═════════╡
│ 314    ┆ LGmLi4R

In [18]:
list(pl_df_f["category_cc"].unique())

['People & Blogs',
 'Pets & Animals',
 'Music',
 'Travel & Events',
 'Science & Technology',
 'Gaming',
 'News & Politics',
 'Entertainment',
 'Autos & Vehicles',
 None,
 'Film and Animation',
 'Howto & Style',
 'Education',
 'Sports',
 'Comedy',
 'Nonprofits & Activism']